In [ ]:
import pandas as pd
import numpy as  np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
test = pd.read_csv('test.csv')
year1 = pd.read_csv('year1.csv')
year2 = pd.read_csv('year2.csv')
year3 = pd.read_csv('year2.csv')
year4 = pd.read_csv('year2.csv')

In [ ]:
year1["year"] = 1
year2["year"] = 2
year3["year"] = 3
year4["year"] = 4
test["year"] = 5

mport pylab as pl
pl.figure(figsize=(12,5))
pl.title("Distribution")
ax = sns.distplot(year1["X1"], color = 'r')

In [ ]:
sns.set(style="white")
# Compute the correlation matrix
corr = all_data.corr()
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 11))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and aspect ratio
sns.heatmap(corr, cmap=cmap, vmax=.5, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

In [ ]:
all_data = pd.concat([year1,year2,year3,year4])

for i in all_data.iloc[:,0:-2]: 
  all_data[i] = pd.to_numeric(all_data[i],errors='coerce')
all_data = all_data.fillna(all_data.mean())

for i in test.iloc[:,0:-1]:
  test[i] = pd.to_numeric(test[i],errors='coerce')
test = test.fillna(test.mean())

def remove_outliers(df, q=0.005):
    upper = df.quantile(1-q)
    lower = df.quantile(q)
    mask = (df < upper) & (df > lower)
    return mask

mask = remove_outliers(all_data)
all_data.iloc[:,0:-2] = all_data[mask]
mask = remove_outliers(test)
test.iloc[:,0:-1] = test[mask]

all_data = all_data.fillna(all_data.mean())
test = test.fillna(test.mean())

X = all_data.iloc[:, 0:-2]
test_X = test.iloc[:,0:-1]
y = all_data.loc[:, 'TARGET']

scaler = StandardScaler()
scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

pca = PCA(n_components = 20)
X_pca = pca.fit_transform(all_data)

distortions = []

for k  in range(1,20):              
    km = KMeans(n_clusters=k,       
                init='k-means++',   
                n_init=10,          
                max_iter=300,       
                random_state=0)     
    km.fit(X_pca)           
    distortions.append(km.inertia_) 

# 結果を散布図に出力
plt.plot(range(1,20), distortions,marker='o') 
plt.xticks([i for i in range(1, 20)])         
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.show()